## Example 1

https://pyabc.readthedocs.io/en/latest/examples/parameter_inference.html

In [1]:
import os
import tempfile
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pyabc
from scipy.stats import uniform, randint

import os
import sys
notebook_dir = os.getcwd()
project_root = os.path.dirname(os.path.dirname(notebook_dir))
sys.path.append(project_root)
from epydemix.calibration import ABCSampler

In [2]:
def model(parameter):
    return {"data": parameter["mu"] + 0.5 * np.random.randn()}

def distance(simulation, data):
    return abs(simulation["data"] - data["data"])

In [3]:
## Pyabc calibration
prior = pyabc.Distribution(mu=pyabc.RV("uniform", 0, 5))
abc = pyabc.ABCSMC(model, prior, distance, population_size=1000)

db_path = os.path.join(tempfile.gettempdir(), "test.db")
observation = 2.5
abc.new("sqlite:///" + db_path, {"data": observation})

#history = abc.run(minimum_epsilon=0.1, max_nr_populations=10)

ABC.Sampler INFO: Parallelize sampling on 8 processes.
ABC.History INFO: Start <ABCSMC id=17, start_time=2025-04-09 22:20:11>


In [4]:
## Epydemix calibration
prior = {"mu": uniform(0, 5)}
abc_epydemix = ABCSampler(model, prior, {}, observation, distance)

results = abc_epydemix.calibrate(strategy="smc", num_generations=10, minimum_epsilon=0.1, num_particles=1000)

Starting ABC-SMC with 1000 particles and 10 generations

Generation 1/10 (epsilon: inf)
	Accepted 1000/1000 (acceptance rate: 100.00%)
	Elapsed time: 00:00:00

Generation 2/10 (epsilon: 1.262998)
Starting SMC generation


KeyboardInterrupt: 

In [ ]:
posterior_pyabc = history.get_distribution()[0]
posterior_epydemix = results.get_posterior_distribution()

sns.kdeplot(posterior_pyabc["mu"], label="Pyabc")
sns.kdeplot(posterior_epydemix["mu"], label="Epydemix")
plt.legend()
plt.show()


In [ ]:
weights_pyabc = history.get_distribution()[1]
weights_epydemix = results.get_weights()    
sns.kdeplot(weights_pyabc, label="Pyabc")
sns.kdeplot(weights_epydemix, label="Epydemix")
plt.legend()
plt.show()

## Example 2

https://pyabc.readthedocs.io/en/latest/examples/discrete_parameters.html

In [11]:
n_data = 100

def model(p):
    return {
        'data': p['p_discrete']
        + np.random.choice([-2, 0, 2], p=[0.2, 0.5, 0.3])
        + p['p_continuous'] * np.random.normal(size=n_data)
    }


distance = lambda simulation, data: sum((simulation['data'] - data['data']) ** 2)
p_true = {'p_discrete': 2, 'p_continuous': 0.5}
obs = model(p_true)

In [ ]:
# domain of possible discrete values
discrete_domain = np.arange(7)

# priors
prior = pyabc.Distribution(
    p_discrete=pyabc.RV('rv_discrete', values=(discrete_domain, [1 / 7] * 7)),
    p_continuous=pyabc.RV('uniform', 0, 2),
)

# transition kernels
transition = pyabc.AggregatedTransition(
    mapping={
        'p_discrete': pyabc.DiscreteJumpTransition(
            domain=discrete_domain, p_stay=0.7
        ),
        'p_continuous': pyabc.MultivariateNormalTransition(),
    }
)

abc = pyabc.ABCSMC(
    model, prior, distance, transitions=transition, population_size=1000
)
abc.new(pyabc.create_sqlite_db_id(), obs)
history = abc.run(max_nr_populations=5)

In [ ]:
prior = {"p_discrete": randint(0, 7), 
         "p_continuous": uniform(0, 2)}
abc_sampler = ABCSampler(model, prior, {}, obs['data'], distance)

results = abc_sampler.calibrate(strategy="smc", num_generations=6, num_particles=1000)

In [ ]:
posterior_pyabc = history.get_distribution()[0]
posterior_epydemix = results.get_posterior_distribution()

sns.histplot(posterior_pyabc["p_discrete"], label="Pyabc")
sns.histplot(posterior_epydemix["p_discrete"], label="Epydemix")
plt.legend()
plt.show()


In [ ]:
sns.kdeplot(posterior_pyabc["p_continuous"], label="Pyabc")
sns.kdeplot(posterior_epydemix["p_continuous"], label="Epydemix")
plt.legend()
plt.show()